## Segmenting and Clustering Neighborhoods in Toronto

### 1. Extract Toronto Neighborhoods Data from Wikipedia Page.

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
r=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").content
soup=BeautifulSoup(r,'lxml')
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"4be8faea-0e68-4c1d-85a9-cdf7c27a3898","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-relat

In [3]:
postal_table=soup.find("table",{"class":"wikitable sortable"})
postal_table.find_all('td')

[<td>M1A
 </td>,
 <td>Not assigned
 </td>,
 <td>Not assigned
 </td>,
 <td>M2A
 </td>,
 <td>Not assigned
 </td>,
 <td>Not assigned
 </td>,
 <td>M3A
 </td>,
 <td>North York
 </td>,
 <td>Parkwoods
 </td>,
 <td>M4A
 </td>,
 <td>North York
 </td>,
 <td>Victoria Village
 </td>,
 <td>M5A
 </td>,
 <td>Downtown Toronto
 </td>,
 <td>Regent Park, Harbourfront
 </td>,
 <td>M6A
 </td>,
 <td>North York
 </td>,
 <td>Lawrence Manor, Lawrence Heights
 </td>,
 <td>M7A
 </td>,
 <td>Downtown Toronto
 </td>,
 <td>Queen's Park, Ontario Provincial Government
 </td>,
 <td>M8A
 </td>,
 <td>Not assigned
 </td>,
 <td>Not assigned
 </td>,
 <td>M9A
 </td>,
 <td>Etobicoke
 </td>,
 <td>Islington Avenue, Humber Valley Village
 </td>,
 <td>M1B
 </td>,
 <td>Scarborough
 </td>,
 <td>Malvern, Rouge
 </td>,
 <td>M2B
 </td>,
 <td>Not assigned
 </td>,
 <td>Not assigned
 </td>,
 <td>M3B
 </td>,
 <td>North York
 </td>,
 <td>Don Mills
 </td>,
 <td>M4B
 </td>,
 <td>East York
 </td>,
 <td>Parkview Hill, Woodbine Gardens
 </td>,


In [4]:
len_table=len((postal_table.find_all('td')))
#extracting Postal code
postal_code=[]
for i in range(1,int(len_table/3)+1):
    postal_code.append(postal_table.find_all('td')[3*i-3].text.replace('\n',""))
postal_code[1:10]

['M2A', 'M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M8A', 'M9A', 'M1B']

In [5]:
#extracting Borough
borough=[]
for i in range(1,int(len_table/3)+1):
    borough.append(postal_table.find_all('td')[3*i-2].text.replace('\n',""))
borough[1:10]

['Not assigned',
 'North York',
 'North York',
 'Downtown Toronto',
 'North York',
 'Downtown Toronto',
 'Not assigned',
 'Etobicoke',
 'Scarborough']

In [6]:
#extracting Neighborhood
neighborhood=[]
for i in range(1,int(len_table/3)+1):
    neighborhood.append(postal_table.find_all('td')[3*i-1].text.replace('\n',""))
neighborhood[1:10]

['Not assigned',
 'Parkwoods',
 'Victoria Village',
 'Regent Park, Harbourfront',
 'Lawrence Manor, Lawrence Heights',
 "Queen's Park, Ontario Provincial Government",
 'Not assigned',
 'Islington Avenue, Humber Valley Village',
 'Malvern, Rouge']

In [7]:
#Combining into a dataframe
import pandas as pd
df=pd.DataFrame(list(zip(postal_code,borough,neighborhood)),columns=['PostalCode','Borough','Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
# Drop the rows when Borough is "Not assigned", i.e. retain all else
df=df[df.Borough!='Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
# Reindex
df.reset_index(drop=True,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df.shape

(103, 3)

### 2. Obtain Latitude and Longitude for each Neighborhood

In [11]:
import geocoder

In [13]:
g=geocoder.google('M1A, Toronto, Ontario')
g.latlng
#Nothing will come out!! I'm choosing to download and use the given CSV file instead.

This didnt work for me either:
```{r}
# initialize your variable to None
lat_lng_coords = None
pc1='M1A'

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(pc1))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
```

In [12]:
coords=pd.read_csv('Geospatial_Coordinates.csv')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
coords.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
#want to retain all postal codes in original dataframe df
df=df.merge(coords,on='PostalCode',how='left')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [29]:
df.shape

(103, 5)

In [15]:
#Check if there are any null values.
df.isnull().sum()

PostalCode      0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

### 3. Explore and Cluster Neighborhoods in Toronto.

In [16]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium

In [17]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto City are 43.6534817, -79.3839347.


We use the function created in the previous lesson to get nearby venues for each postal code.

In [18]:
CLIENT_ID = 'A2Y0GH2SUYTTVLDESYA12MBQCMXTPMPPQR3BCO1GAWYNNQ1B' # your Foursquare ID
CLIENT_SECRET = '0E0U5OVTVY1QUKNIC0KM0LVKN0PKVLWMFZXR2VLGI45YC2DX' # your Foursquare Secret
VERSION = '20190617' # Foursquare API version

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
LIMIT=50
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(1673, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [22]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 252 unique categories.


In [25]:
# Turn the categories into columns for each Neighborhood
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix="") #prefix parameters to remove word "Venue"

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,_Accessories Store,_Airport,_Airport Food Court,_Airport Lounge,_Airport Service,_Airport Terminal,_American Restaurant,_Antique Shop,_Aquarium,...,_Vegetarian / Vegan Restaurant,_Video Game Store,_Video Store,_Vietnamese Restaurant,_Warehouse Store,_Wine Bar,_Wine Shop,_Wings Joint,_Women's Store,_Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
toronto_onehot.shape

(1673, 253)

In [27]:
#Group by Neighborhood
toronto_grouped=toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped.head()

,Neighborhood,_Accessories Store,_Airport,_Airport Food Court,_Airport Lounge,_Airport Service,_Airport Terminal,_American Restaurant,_Antique Shop,_Aquarium,...,_Vegetarian / Vegan Restaurant,_Video Game Store,_Video Store,_Vietnamese Restaurant,_Warehouse Store,_Wine Bar,_Wine Shop,_Wings Joint,_Women's Store,_Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
toronto_grouped.shape

(94, 253)

In [30]:
#print top 5 venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                        venue  freq
0             _Breakfast Spot  0.25
1                     _Lounge  0.25
2  _Latin American Restaurant  0.25
3               _Skating Rink  0.25
4          _Accessories Store  0.00


----Alderwood, Long Branch----
             venue  freq
0     _Pizza Place  0.29
1     _Coffee Shop  0.14
2             _Gym  0.14
3  _Sandwich Place  0.14
4             _Pub  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                  venue  freq
0                 _Bank  0.11
1          _Coffee Shop  0.11
2          _Pizza Place  0.05
3  _Fried Chicken Joint  0.05
4        _Shopping Mall  0.05


----Bayview Village----
                  venue  freq
0  _Japanese Restaurant  0.25
1                 _Bank  0.25
2   _Chinese Restaurant  0.25
3                 _Café  0.25
4        _Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                      venue  freq
0           _Sandwich Place  0.09
1              _Coffee Shop

In [31]:
#Function to sort values then get top n venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [120]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) #for 1-3
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) #for 4 onwards

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
# fill in Neighborhood column
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

# fill in output of function per row
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,_Breakfast Spot,_Lounge,_Latin American Restaurant,_Skating Rink,_Yoga Studio,_Diner,_Ethiopian Restaurant,_Electronics Store,_Eastern European Restaurant,_Dumpling Restaurant
1,"Alderwood, Long Branch",_Pizza Place,_Coffee Shop,_Sandwich Place,_Athletics & Sports,_Pub,_Gym,_Concert Hall,_Dessert Shop,_Electronics Store,_Eastern European Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",_Coffee Shop,_Bank,_Shopping Mall,_Sandwich Place,_Fried Chicken Joint,_Ice Cream Shop,_Diner,_Mobile Phone Shop,_Sushi Restaurant,_Restaurant
3,Bayview Village,_Café,_Japanese Restaurant,_Bank,_Chinese Restaurant,_Doner Restaurant,_Discount Store,_Distribution Center,_Dog Run,_Yoga Studio,_Diner
4,"Bedford Park, Lawrence Manor East",_Coffee Shop,_Restaurant,_Sandwich Place,_Italian Restaurant,_Pizza Place,_Thai Restaurant,_Pharmacy,_Pub,_Café,_Butcher


In [121]:
# Clean output
neighborhoods_venues_sorted.replace("_","",regex=True,inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(94, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Latin American Restaurant,Skating Rink,Yoga Studio,Diner,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Athletics & Sports,Pub,Gym,Concert Hall,Dessert Shop,Electronics Store,Eastern European Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Shopping Mall,Sandwich Place,Fried Chicken Joint,Ice Cream Shop,Diner,Mobile Phone Shop,Sushi Restaurant,Restaurant
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Yoga Studio,Diner
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Pizza Place,Thai Restaurant,Pharmacy,Pub,Café,Butcher


Cluster Neighborhoods using K-means clustering where similarity is based on top venues.

In [122]:
# set number of clusters
# when cluster=5, clusters 3 and 5 are similar
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [123]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [124]:
# merge toronto_grouped with initial toronto data(df) to add latitude/longitude for each neighborhood
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')
print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(98, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Construction & Landscaping,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,French Restaurant,Drugstore,Dumpling Restaurant,Donut Shop,Eastern European Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Yoga Studio,Beer Store,Ice Cream Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Accessories Store,Vietnamese Restaurant,Miscellaneous Shop,Event Space,Coffee Shop,Furniture / Home Store,Boutique,Coworking Space,Ethiopian Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Sandwich Place,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Italian Restaurant


In [125]:
# create map to visualize results
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Check clusters

#### Cluster 1

In [126]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
#Lots of Coffeeshops and Restaurants

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,French Restaurant,Drugstore,Dumpling Restaurant,Donut Shop,Eastern European Restaurant
2,Downtown Toronto,0,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Yoga Studio,Beer Store,Ice Cream Shop
3,North York,0,Clothing Store,Accessories Store,Vietnamese Restaurant,Miscellaneous Shop,Event Space,Coffee Shop,Furniture / Home Store,Boutique,Coworking Space,Ethiopian Restaurant
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Sandwich Place,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
7,North York,0,Gym,Coffee Shop,Beer Store,Japanese Restaurant,Asian Restaurant,Restaurant,Italian Restaurant,Supermarket,Café,Sandwich Place
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Gym,Hotel,Seafood Restaurant,Tea Room,Deli / Bodega,Concert Hall,American Restaurant
98,Etobicoke,0,Pool,River,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
99,Downtown Toronto,0,Coffee Shop,Restaurant,Yoga Studio,Sushi Restaurant,Japanese Restaurant,Men's Store,Burger Joint,Bubble Tea Shop,Gay Bar,Ethiopian Restaurant
100,East Toronto,0,Brewery,Burrito Place,Restaurant,Fast Food Restaurant,Recording Studio,Auto Workshop,Light Rail Station,Farmers Market,Garden Center,Garden


#### Cluster 2

In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
# Surrounded by parks

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Construction & Landscaping,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
10,North York,1,Sushi Restaurant,Japanese Restaurant,Park,Pub,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
16,York,1,Hockey Arena,Field,Park,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
21,York,1,Park,Women's Store,Pool,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
32,Scarborough,1,Playground,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
35,East York,1,Park,Convenience Store,Metro Station,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
49,North York,1,Bakery,Construction & Landscaping,Park,Basketball Court,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
61,Central Toronto,1,Park,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
64,York,1,Convenience Store,Park,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
66,North York,1,Convenience Store,Park,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


#### Cluster 3

In [128]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
# Event space

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,2,Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant


#### Cluster 4

In [129]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
# Baseball fields

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,3,Baseball Field,Yoga Studio,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
101,Etobicoke,3,Home Service,Baseball Field,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
